##### tasks

<ol>
<li>for various time slices, for each planet in each run, parse for/calculate these values
<ol>
<li>eccentricity, jacobi distance, pericenter </li>
</ol>
<ol>
<li>for nearest and second nearest neighbor: ratios of semi-major axis, mutual hill radii, mass </li>
</ol>
</li>
<li>over the duration tracked (3, 5, 10, 20, 100 timesteps), for each planet in each run, find these values
<ol>
<li>max, min, average, standard deviation for each value</li>
</ol>
</li>
<li>write into a csv file. separate file for each time slice. include run and planet numbers for easy tracking</li>
<li>looking at how they change over time for each planet and system to tell which initial conditions might be most telling</li>
</ol>

###### initial code, using different sets of variables for inner and outer planets -- can ignore
(mostly for testing)

In [ ]:
import pickle
import pandas as pd
import numpy as np


data = dict()

cols = ['Time','mass','a','e','i','omega','Omega','M','dist_jac','dist_cen','x','y','z','u','v','w']


for rn in range(1,2):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = pd.read_csv(r'/Users/neha/Documents/lanl/classifier/run'+str(rn)+'/PL'+str(planetnum)+'.el', skiprows=2, delimiter=r"\s+", names=cols) 

        for time in range(0,3):
            data[rn][planetnum][time] = dict()

            for col in cols:
                data[rn][planetnum][time][col] = dict()
                try:
                    data[rn][planetnum][time][col] = pl[col][time]
                except:
                    pass

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  

varA = ['e','pericenter','jacobi','mhrC','mhrF','aC_ratio','aF_ratio','massC_ratio','massF_ratio'] # C = closer, F = farther
varB = ['e','pericenter','jacobi','mhrI','mhrO','aI_ratio','aO_ratio','massI_ratio','massO_ratio'] # I = inner, O = outer

    
for rn2 in range(1,2):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['e'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        
        if (planet == 1 or planet == 10):
            te[rn2][planet]['mhrC'] = []
            te[rn2][planet]['mhrF'] = []
            te[rn2][planet]['aC_ratio'] = []
            te[rn2][planet]['aF_ratio'] = []
            te[rn2][planet]['massC_ratio'] = []
            te[rn2][planet]['massF_ratio'] = []
            
        else:
            te[rn2][planet]['mhrI'] = []
            te[rn2][planet]['mhrO'] = []
            te[rn2][planet]['aI_ratio'] = []
            te[rn2][planet]['aO_ratio'] = []
            te[rn2][planet]['massI_ratio'] = []
            te[rn2][planet]['massO_ratio'] = []

        
        
        for timestep in range (0,3):   
            try:
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['pericenter'].append(float(data[rn2][planet][timestep]['omega']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['dist_jac']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                

                if (planet == 1 or planet == 10):
                    if (planet == 1):
                        m1 = float(data[rn2][planet+1][timestep]['mass'])
                        a1 = float(data[rn2][planet+1][timestep]['a'])

                        m2 = float(data[rn2][planet+2][timestep]['mass'])
                        a2 = float(data[rn2][planet+2][timestep]['a'])

                    else:
                        m1 = float(data[rn2][planet-1][timestep]['mass'])
                        a1 = float(data[rn2][planet-1][timestep]['a'])

                        m2 = float(data[rn2][planet-2][timestep]['mass'])
                        a2 = float(data[rn2][planet-2][timestep]['a'])

                    te[rn2][planet]['mhrC'].append((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                    te[rn2][planet]['mhrF'].append(float((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2)))
                    te[rn2][planet]['aC_ratio'].append(float(aPL/a1))
                    te[rn2][planet]['aF_ratio'].append(float(aPL/a2))
                    te[rn2][planet]['massC_ratio'].append(float(massPL/m1))
                    te[rn2][planet]['massF_ratio'].append(float(massPL/m2))


                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                    te[rn2][planet]['mhrI'].append(float(((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                    te[rn2][planet]['mhrO'].append(float(((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                    te[rn2][planet]['aI_ratio'].append(float(aPL/a1))
                    te[rn2][planet]['aO_ratio'].append(float(aPL/a2))
                    te[rn2][planet]['massI_ratio'].append(float(massPL/m1))
                    te[rn2][planet]['massO_ratio'].append(float(massPL/m2))
                    
            except:
                pass

            
            
for rn2 in range(1,2):
    for planet in range (1,11):
        if (planet == 1 or planet == 10):
            for vA in varA:
                sys[rn2][planet]['initial ' + str(vA)] = te[rn2][planet][str(vA)][0]
                sys[rn2][planet]['final ' + str(vA)] = te[rn2][planet][str(vA)][-1]
                sys[rn2][planet]['mean ' + str(vA)] = np.mean(te[rn2][planet][str(vA)])  
                sys[rn2][planet]['median ' + str(vA)] = np.median(te[rn2][planet][str(vA)])
                sys[rn2][planet]['min ' + str(vA)] = np.amin(te[rn2][planet][str(vA)])
                sys[rn2][planet]['max ' + str(vA)] = np.amax(te[rn2][planet][str(vA)])
                sys[rn2][planet]['sd ' + str(vA)] = np.std(te[rn2][planet][str(vA)])
        else:
            for vB in varB:
                sys[rn2][planet]['initial ' + str(vB)] = te[rn2][planet][str(vB)][0]
                sys[rn2][planet]['final ' + str(vB)] = te[rn2][planet][str(vB)][-1]
                sys[rn2][planet]['mean ' + str(vB)] = np.mean(te[rn2][planet][str(vB)])  
                sys[rn2][planet]['median ' + str(vB)] = np.median(te[rn2][planet][str(vB)])
                sys[rn2][planet]['min ' + str(vB)] = np.amin(te[rn2][planet][str(vB)])
                sys[rn2][planet]['max ' + str(vB)] = np.amax(te[rn2][planet][str(vB)])
                sys[rn2][planet]['sd ' + str(vB)] = np.std(te[rn2][planet][str(vB)])
           
                
                
for rn2 in range(1,2):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                if "ejected" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int((cols[4])[2])]['end time'] = float(cols[6])

    
    
print(sys)

###### standardized variables for all planets - used for data collection
(for planets 1 & 10, two neighboring planets. for rest, inner and outer planets)
only used initial data -- did not account for planet orbits & planet neighbors changing

In [ ]:
import pandas as pd
import numpy as np
import csv


data = dict()

cols = ['Time','mass','a','e','i','omega','Omega','M','dist_jac','dist_cen','x','y','z','u','v','w']


for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = pd.read_csv(r'/Users/neha/Documents/lanl/classifier/run'+str(rn)+'/PL'+str(planetnum)+'.el', skiprows=2, delimiter=r"\s+", names=cols) 

        for time in range(0,100):
            data[rn][planetnum][time] = dict()

            for col in cols:
                data[rn][planetnum][time][col] = dict()
                try:
                    data[rn][planetnum][time][col] = pl[col][time]
                except:
                    pass

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
var = ['e','pericenter','jacobi','mhr1','mhr2','a1_ratio','a2_ratio','mass1_ratio','mass2_ratio'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['e'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['a1_ratio'] = []
        te[rn2][planet]['a2_ratio'] = []
        te[rn2][planet]['mass1_ratio'] = []
        te[rn2][planet]['mass2_ratio'] = []

    
        for timestep in range (0,100):
            try:
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['dist_jac']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                ePL = float(data[rn2][planet][timestep]['e'])
                
                if (planet == 1):
                    m1 = float(data[rn2][planet+1][timestep]['mass'])
                    a1 = float(data[rn2][planet+1][timestep]['a'])

                    m2 = float(data[rn2][planet+2][timestep]['mass'])
                    a2 = float(data[rn2][planet+2][timestep]['a'])

                elif (planet == 10):
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet-2][timestep]['mass'])
                    a2 = float(data[rn2][planet-2][timestep]['a'])
                        
                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                    
                te[rn2][planet]['mhr1'].append((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                te[rn2][planet]['mhr2'].append(float((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2)))
                te[rn2][planet]['a1_ratio'].append(float(aPL/a1))
                te[rn2][planet]['a2_ratio'].append(float(aPL/a2))
                te[rn2][planet]['mass1_ratio'].append(float(massPL/m1))
                te[rn2][planet]['mass2_ratio'].append(float(massPL/m2))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

            
            
for rn2 in range(1,101):
    for planet in range (1,11):
        for v in var:
            sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
            sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
            sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
            sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
            sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
            sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
           
                

# instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                if "ejected" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int((cols[4])[2])]['end time'] = float(cols[6])
                    

# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                if "ejected" in line:
                    sys[rn2][int((cols[0])[2])]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][int((cols[0])[2])]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int((cols[4])[2])]['fate'] = "planet-planet collision"

###### writing data into csv file

In [ ]:
nested_keys = []

var = ['e','pericenter','jacobi','mhr1','mhr2','a1_ratio','a2_ratio','mass1_ratio','mass2_ratio'] 

for v in var:
    nested_keys.append('initial ' + str(v))
    nested_keys.append('final ' + str(v))
    nested_keys.append('mean ' + str(v))
    nested_keys.append('min ' + str(v))
    nested_keys.append('max ' + str(v))
    nested_keys.append('sd ' + str(v))
    


with open('100ts.csv', 'w') as f:
    header = 'run, planet, ' + ', '.join(map(str,nested_keys)) + ', end time, fate' + '\n'
    f.write(header)
    
    for r in list(sys.keys()):
        for p in list(sys[r].keys()):
            line = ''
            line += str(r) + ', ' + str(p)
            for va in nested_keys:
                line += ', ' + str(sys[r][p][va])
            line += ', ' + str(sys[r][p]['end time']) + ', ' + str(sys[r][p]['fate'])
            line += '\n'
            f.write(line)